# 3. Dutch Auction Model

---

## Model Introduction

### Purpose

This model is meant to estimate the (monetary) value the JASIRI token will settle at after the Dutch Auction.


### Questions

1. What price will the JASIRI token settle at after the Dutch Auction?
2. How does this price change with change in bidder characteristics, number, and JASIRI characteristics.


### Assumptions

1. Bidders are completely unaware of bids made by other bidders.


### Constraints / Scope

* Bidder characteristics are vaguely defined due to insifficient data.


### Model input

1. The following quantifiable metrics of the afropreneur landscape: ${N}$, ${\mu}$, ${\sigma}$.

2. Market conditions.


### Model description



### Shortcomings of the model

${n}$, the number of afropreneurs per range, is treated as constant in the model. It should eventually be variable subject to the afropreneur landscape that will be defined and refined later, and the varied response to different market conditions along of the spectrum of these afropreneurs.

## 0. Dependencies

In [35]:
import math
import pandas as pd
from scipy.integrate import quad
from scipy.stats import norm

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

## 1. State Variables

In [36]:
initial_state = {
    'market_cap': 0
}

# 2. System Parameters

> These are variables that impact the behaviour of the model. They allow consideration of how the system behaves under a different model parameter set (and different parameter values).

In [37]:
system_params = {
    'num_afropreneurs': [600000],
    'avg_investable_capital': [30],  # all values in US dollars, unless stated otherwise
    'std_dev_investable_capital': [20]
    #'market_conditions': {}
}

# 3. Policy Functions

> A Policy Function computes one or more signals to be passed to State Update Functions. They describe the logic  and behaviour of a system component  or mechanism.

In [38]:
# utility functions: functions that other constructs in the model make use of, and are abstracted away for simplicity
def normal_dist(x):
    return x * norm.pdf(x, system_params['avg_investable_capital'], system_params['std_dev_investable_capital'])


def p_purchase_simulation(params, substep, state_history, previous_state):
    purchase = 0
    commitment_range = len(range(params['avg_investable_capital'][0]-(2*params['std_dev_investable_capital'][0]),
                             params['avg_investable_capital'][0]+(2*params['std_dev_investable_capital'][0])))
    afropren_batch_size = 1000  # f(market states)
    cash_batch_size = commitment_range / (600000 / afropren_batch_size)
    lower_cash_value = 0
    for i in range(math.ceil(600000 / afropren_batch_size)):
        purchase += afropren_batch_size * quad(normal_dist, lower_cash_value, lower_cash_value + cash_batch_size)[0]
        lower_cash_value += cash_batch_size
    return {'delta_purchase': purchase}


print(p_purchase_simulation(system_params, 2, [], {}))

{'delta_purchase': 30049.279905607465}


# 4. State Update Functions

> We create State Update Functions to design the way our model state changes over time.

In [53]:
def s_update_market_cap(params, substep, state_history, previous_state, policy_input):
    global market_cap
    market_cap = previous_state['market_cap'] + policy_input['delta_purchase']
    return market_cap

In [54]:
# Relevant state variables
market_cap = initial_state['market_cap']

In [55]:
next_state = {
    'market_cap': s_update_market_cap(system_params, 2, [], initial_state, p_purchase_simulation(system_params, 2, [], initial_state)),
}
next_state

{'market_cap': 30049.279905607465}

# 5. Partial State Update Blocks
## Tying it all together

> A series of Partial State Update Blocks is a structure for composing State Update Functions and Policy Functions in series or parallel, as a representation of the system model. 

**Updates run in parallel**

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {
        },
        'variables': {
            'market_cap': s_update_market_cap
        }
    }
]

# 6. Configuration

> The configuration stage is about tying all the previous model components together and choosing how the simulation should run.

<center>
<img src="./images/cadcad-flow.png"
     alt="cadCAD flow"
     style="width: 25%;" />
</center>

Configuration parameters:
* `'N': 1` - the number of times we'll run the simulation (you'll see them called "Monte Carlo runs" later in the course, when we look at tools to analyze system models)
* `'T': range(400)` - the number of timesteps the simulation will run for
* `'M': system_params` - the parameters of the system

In [43]:
sim_config = config_sim({
    "N": 1,
    "T": range(1),
    "M": system_params
})

In [44]:
from cadCAD import configs
del configs[:] # Clear any prior configs

In [45]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

# 7. Execution

> The Execution Engine takes a model and configuration, and computes the simulation output.

## Configuring the cadCAD simulation execution

In [46]:
exec_context = ExecutionContext()

In [47]:
simulation = Executor(exec_context=exec_context, configs=configs)

## Time to simulate our ecosystem model!

In [57]:
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (1, 3, 1, 1)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded


KeyError: 'delta_purchase'

# 8. Simulation Output Preparation
> The simulation results are returned as a list of Python dictionaries, which we then convert to a Pandas dataframe. At this stage of the process we manipulate and analyze our results to answer questions about our model.

In [ ]:
simulation_result = pd.DataFrame(raw_result)

In [ ]:
raw_result[:1]

In [ ]:
simulation_result.head()

# 9. Simulation Analysis

In [ ]:
pd.options.plotting.backend = "plotly"

After plotting the results, we update the parameters and then select run all cells above.

In [56]:
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['market_cap']
)

NameError: name 'simulation_result' is not defined

In [ ]:
pd.set_option('display.max_rows', len(simulation_result))
display(simulation_result)
pd.reset_option('display.max_rows')